Recommendation de films basé sur le genre , les thèmes et le vote


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

Charger les données grace à pandas

In [3]:
donnees = pd.read_csv("dataset movies imdb\TMDB_movie_dataset_v11.csv")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Freud Arthur\AppData\Local\Temp\ipykernel_3012\81128966.py:1: SyntaxWarning: invalid escape sequence '\T'
  donnees = pd.read_csv("dataset movies imdb\TMDB_movie_dataset_v11.csv")


In [4]:
print(f" Les colonnes du dataset sont {donnees.columns}")


 Les colonnes du dataset sont Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')


Le scaler et la vectorisation

In [5]:
scaler = MinMaxScaler()
vectorizer = TfidfVectorizer()
donnees['combined'] = donnees['genres'].fillna('') + " " + donnees['keywords'].fillna('')

genre_and_keywords = donnees[['genres', 'keywords']]
genre_and_keywords_vectoriser = vectorizer.fit_transform(donnees['combined'])
votes = donnees[['vote_average']]

votes_scaler = scaler.fit_transform(votes)


In [47]:
def suggestions_de_films(film):

    for a in donnees['title'].dropna():
        if a.strip().lower() == film.lower():
            print("Trouvé")
            break
    else:
        print("Film pas trouvé ... Veuillez réessayer")
        return


    recommendation =[]
    genre_film = donnees.loc[donnees['title'].str.lower() == film.lower() , ['genres', 'keywords']].iloc[0]
    texte_film = genre_film['genres'] + " " + genre_film['keywords']
    genre_film_vectoriser = vectorizer.transform([texte_film])

    #for index , f in enumerate(genre_and_keywords_vectoriser):
    score  = cosine_similarity(genre_film_vectoriser , genre_and_keywords_vectoriser).flatten()
    
        
    recommendation = list(zip(donnees['title'] , score, donnees['vote_average']))

    recommendation.sort(key=lambda x: (x[1]), reverse=True)

    top_5 = recommendation[:5]
    return recommendation[:5]

        

        

In [7]:
film = donnees.loc[donnees['title'] == 'Django Unchained' , ['genres', 'keywords']]
film

,genres,keywords
13,"Drama, Western","rescue, friendship, bounty hunter, texas, slav..."
702601,NaN,NaN


In [52]:
suggestions_de_films("Avengers: Endgame")
print(f"Les films recommandes pour Iron Man 3 sont :{',' .join ({t[0] for t in suggestions_de_films("doctor ")} if suggestions_de_films("doctor ") else 'Aucun film trouvé')}")



Trouvé
Film pas trouvé ... Veuillez réessayer
Les films recommandes pour Iron Man 3 sont :A,u,c,u,n, ,f,i,l,m, ,t,r,o,u,v,é


In [10]:
b = [donnees['title'] == 'Interstellar']
b

[0          False
 1           True
 2          False
 3          False
 4          False
            ...  
 1268298    False
 1268299    False
 1268300    False
 1268301    False
 1268302    False
 Name: title, Length: 1268303, dtype: bool]

In [33]:
donnees.groupby('genres')['vote_average'].mean().sort_values()
donnees.sort_values(by='vote_average', ascending=False).head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,combined
287695,547278,Raw Luck,10.0,1,Released,2005-04-01,0,102,True,/v7UFTzprmsl1nYTRdImX94ajP5t.jpg,...,Introducing Raw and their debut film Raw Luck!...,0.600,/uIjaXg8F6Y42j09YjGaQ3XWxYaR.jpg,NaN,NaN,"AVI Production, RAW, Eurocreme, Staxus",Czech Republic,Czech,"twink, bareback, gay pornography","twink, bareback, gay pornography"
269084,800644,Three stories and a crossword,10.0,1,Released,2019-12-02,0,20,False,/gGxlecY4A1wBZaipX0BC5w2mgSO.jpg,...,"In a nursing home, a word in a crossword, an u...",0.883,/81cfmpFhORoVf1rz5G5tIJVv4MJ.jpg,NaN,"Drama, Comedy",NaN,Argentina,Spanish,NaN,"Drama, Comedy"
324616,951178,hololive 3rd fes. Link Your Wish Day 1,10.0,1,Released,2022-03-19,0,205,False,/uOqVBzlGRMmWb4ZxgXqxhdNQEnw.jpg,...,hololive 3rd fes. Link Your Wish is a live con...,0.667,/p6nOcztGpbEZ9I6YzixhPWzOEcB.jpg,hololive 3rd fes. Link Your Wish Day 1,"Animation, Comedy, Music","COVER Corp., hololive production, Makuhari Media",Japan,"Japanese, English","cover-corp, holoen","Animation, Comedy, Music cover-corp, holoen"
239240,434474,That Guy and I,10.0,1,Released,1961-09-10,0,105,False,/6AILwAvRQAgdhPsJJT1STIylto0.jpg,...,Asada Keiko and Kurokawa Saburo are two young ...,1.309,/eWJEs3H3FwrslySEHqZgguElrb6.jpg,NaN,"Romance, Comedy",Nikkatsu Corporation,Japan,Japanese,NaN,"Romance, Comedy"
239237,471411,Femaliens: Seduction of the Species,10.0,1,Released,2017-08-15,0,101,False,NaN,...,"The Femaliens have touched down, and they want...",1.115,/mCnU0mi9omtcnCq2akcCJWILrJM.jpg,Ground Zero: Venice Beach. The invasion of hot...,"Science Fiction, Fantasy",Surrender Cinema,NaN,English,softcore,"Science Fiction, Fantasy softcore"
324622,1500490,Making Taro Talk: A Documentary,10.0,1,Released,2025-05-23,0,48,False,NaN,...,A group of undergraduate students at UC San Di...,0.000,/3K3nxU0aFnSlMahtIE38U715FJc.jpg,Countless wires and cardboard. Zero time. One ...,"Documentary, Drama, Comedy",Themed Entertainment Association at UCSD,United States of America,English,"friendship, animatronic","Documentary, Drama, Comedy friendship, animatr..."
239232,725122,Selbst Geheilt,10.0,1,Released,2020-04-21,0,0,False,NaN,...,NaN,0.600,/eDdNOt2NDPj4KqOxR0sUFapLRiG.jpg,NaN,Documentary,InTouch Filmproduktion Stephan Petrowitsch,Germany,German,NaN,Documentary
269081,599959,Heatstroke,10.0,1,Released,2019-05-04,0,9,False,NaN,...,“Heatstroke” reflects on this generation’s sex...,0.600,/cNhLOM1mjMvr4Ciz7vMIMMcSWE6.jpg,NaN,Drama,Volpana,"Portugal, United States of America",English,short film,Drama short film
239228,714318,Bob Geldof - Berlin Live,10.0,1,Released,2015-06-10,0,0,False,NaN,...,NaN,0.600,/bqH9a2T7g47wKtKlQMdDBusg23L.jpg,NaN,Music,NaN,NaN,English,NaN,Music
239227,1420889,Electric Blue,10.0,1,Released,2025-01-24,0,0,False,/sQmGPSNeqZWldJgLiMddf0Bbkmt.jpg,...,When a runaway thinks she has been found out s...,1.400,/2AGRYY3dQ8x7H5fYwT1KNCk6jAT.jpg,I was always going to leave,Drama,NaN,Netherlands,NaN,NaN,Drama


In [36]:
donnees.fillna('')

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,combined
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.9520,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...","Action, Science Fiction, Adventure rescue, mis..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,The adventures of a group of explorers who mak...,140.2410,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...","Adventure, Drama, Science Fiction rescue, futu..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.6430,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...","Drama, Action, Crime, Thriller joker, sadism, ..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.9320,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...","Action, Adventure, Fantasy, Science Fiction fu..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.0820,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...","Science Fiction, Action, Adventure new york ci..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268298,797301,The Cloud,0.000,0,Released,2014-03-07,0,21,False,,...,"Ana lives in a country town, a single mother w...",0.6000,/4fTFeBv4JUN9ed98opQDsOfU0rT.jpg,,Drama,,Cuba,Spanish,,Drama
1268299,797302,Round and Brown 50,0.000,0,Released,2021-02-12,0,90,True,,...,"StarringAaliyah Hadid , Harley Dean , Moriah M...",0.6000,/vcUdAV9keb6Mq12odkZMJAp6pmP.jpg,,,Reality Kings,United States of America,English,,
1268300,797303,Hidden Talents,0.000,0,Released,1986-11-15,0,0,False,,...,Nellie (Pat Phoenix) is weak and bedridden in ...,0.8430,/hWAURQ6LY1NBbE7GDWaLuclQsjc.jpg,,,,,,,
1268301,797304,Il ritratto incompiuto,0.000,0,Released,2019-11-02,0,0,False,,...,,0.8610,/q31EITtc7EQxQZYuKzixIYiNAWc.jpg,,,,,,,


In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [5]:
import pandas as pd

# Lire directement depuis le lien IMDb
url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(url, compression='gzip', sep='\t')

print(ratings)


            tconst  averageRating  numVotes
0        tt0000001            5.7      2182
1        tt0000002            5.5       302
2        tt0000003            6.4      2256
3        tt0000004            5.2       194
4        tt0000005            6.2      2995
...            ...            ...       ...
1629027  tt9916846            5.3         7
1629028  tt9916848            5.2         7
1629029  tt9916850            6.0         7
1629030  tt9916852            5.7         7
1629031  tt9916880            7.7        10

[1629032 rows x 3 columns]


In [12]:
import kagglehub
from pathlib import Path

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata" , path=Path.cwd())
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata/versions/2
The server reported the following issues: Dataset not found
Please make sure you specified the correct resource identifiers.